In [838]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")

D=6;
chi=10;
W=5;
N=4;
tol=1e-6;

pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [839]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);

@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_bond");


"initialize CTM"

"start CTM iterations:"

  0.099842 seconds (589.26 k allocations: 64.480 MiB)
CTMRG iteration: 1, CTMRG err: 2.381843388312667
CTMRG iteration: 2, CTMRG err: 0.16758887210702916
CTMRG iteration: 3, CTMRG err: 0.01596027742626989
CTMRG iteration: 4, CTMRG err: 0.005665817296959171
CTMRG iteration: 5, CTMRG err: 0.0016753918020690128
CTMRG iteration: 6, CTMRG err: 0.0006074576485720598
CTMRG iteration: 7, CTMRG err: 0.00023375102816346817
CTMRG iteration: 8, CTMRG err: 9.659154953051407e-5
CTMRG iteration: 9, CTMRG err: 4.080785607776493e-5
CTMRG iteration: 10, CTMRG err: 1.7562204063620438e-5
CTMRG iteration: 11, CTMRG err: 7.617736490288161e-6
CTMRG iteration: 12, CTMRG err: 3.3195128247064078e-6
CTMRG iteration: 13, CTMRG err: 1.4495809776378003e-6
CTMRG iteration: 14, CTMRG err: 6.336851062992582e-7
  2.925620 seconds (10.73 M allocations: 1020.137 MiB, 7.81% gc time, 18.33% compilation time)
  1.287662 seconds (4.94 M allocations: 549.014 MiB, 8.21% gc time, 46.53% compilation time)
  0.847608 seconds (3.7

In [840]:
display((E_up+E_down)/3)
Tleft=CTM["Tset"][4];
Tright=CTM["Tset"][2];

@tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
@tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];


-0.4272664643496391 + 6.7966121702560844e-18im

In [841]:
function HV_L_tensor(vl,A,mpo)
    if mpo==[]
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A'[3,-2,2]*A[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A'[3,-2,2]*A[1,-3,2];
        end
    else
        if numind(vl)==5
            @tensor vl[:]:=vl[-1,7,5,3,1]*A'[7,-2,6]*mpo'[5,6,-3,4]*mpo[3,2,-4,4]*A[1,-5,2];
        elseif numind(vl)==4
            @tensor vl[:]:=vl[7,5,3,1]*A'[7,-1,6]*mpo'[5,6,-2,4]*mpo[3,2,-3,4]*A[1,-4,2];
        end
    end
    return vl
end

function HV_R_tensor(vr,A,mpo)
    if mpo==[]
        if numind(vr)==3
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
        elseif numind(vr)==2
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3];
        end
    else
        if numind(vr)==5
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1,-5];
        elseif numind(vr)==4
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1];
        end
    end
    return vr
end

function left_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_L_tensor(x,A,[]);
        vl_init = permute(TensorMap(randn, space(A,1), space(A,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_L_tensor(x,A,mpo);
        vl_init = permute(TensorMap(randn, space(A,1)*space(mpo,1),space(mpo,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end

function right_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_R_tensor(x,A,[]);
        vr_init = permute(TensorMap(randn, space(A,2), space(A,2)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_R_tensor(x,A,mpo);
        vr_init = permute(TensorMap(randn, space(A,2)*space(mpo,3),space(mpo,3)*space(A,2)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end



function left_eigenvalue(A1,A2,n)
    function HV_L_A1A2(vl,A1,A2)
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A1'[3,-2,2]*A2[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A1'[3,-2,2]*A2[1,-3,2];
        end
    return vl
    end
    HVA1A2fun(x)=HV_L_A1A2(x,A1,A2);
    vl_init = permute(TensorMap(randn, space(A1,1), space(A2,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
    eu,ev=eigsolve(HVA1A2fun, vl_init, 1,:LM,Arnoldi());
    @assert maximum(abs.(eu)) == abs(eu[1])
    eu=eu[1:n];
    return eu
end

left_eigenvalue (generic function with 1 method)

In [842]:
function impo_imps(mpo,A)
    unitary_mpo_A=unitary(fuse(space(mpo,1)⊗space(A,1)), space(mpo,1)⊗space(A,1));
    @tensor mpo_A[:]:=unitary_mpo_A[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_mpo_A'[4,5,-2];
    return mpo_A
end

impo_imps (generic function with 1 method)

In [843]:
function itebd_ite(mpo,A,W,type)
    if type=="A"
        mpo_A=impo_imps(mpo,A);
        vl,_=left_eigenvector(mpo_A,[],"A");
        eu,ev=eig(permute(vl,(1,),(2,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,1]*mpo_A[1,2,-3]*unitary_R[2,-2];
    elseif type=="mpo_A"
        vl,_=left_eigenvector(A,mpo,"mpo_A");
        eu,ev=eig(permute(vl,(2,1,),(3,4,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_R[4,5,-2];
    end

    vr,_=right_eigenvector(AL,[],"A");

    U,S,_=tsvd(permute(vr,(1,),(2,)),trunc=truncdim(W));
    #display(diag(convert(Array,S)))
    @tensor A_trun[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];

    return A_trun
end


itebd_ite (generic function with 1 method)

In [844]:
function left_right_normalize(A)
    #left normalize
    vl,_=left_eigenvector(A,[],"A");
    eu,ev=eig(permute(vl,(1,),(2,)));
    unitary_L=sqrt(eu)*ev';
    unitary_R=ev*pinv(sqrt(eu));
    @tensor AL[:]:=unitary_L[-1,1]*A[1,2,-3]*unitary_R[2,-2];
    #transform right fixed point to be diagonal
    vr,eu=right_eigenvector(AL,[],"A");
    U,S,_=tsvd(permute(vr,(1,),(2,)));
    @tensor AL[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];
    AL=AL/sqrt(eu);
    return AL
end


left_right_normalize (generic function with 1 method)

In [846]:
function ITEBD_boundary_groundstate(O1,O2,W,A_init,method)
# ITEBD_boundary_groundstate(O1,O2,D,inv_tol):
    #use power method to obtain dominant boundary imps of nonhermitian MPO
    if method=="OO"
        @tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
        U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
        @tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
    end
    #Initial state
    if A_init==[]
        mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
        A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());
    else 
    end
    AL=left_right_normalize(A_init);
    
    for cp=1:30
        AL_old=AL;
        @time if method=="O_O"
            A_trun_intermed=itebd_ite(O1,AL,3*W,"mpo_A");
            A_trun=itebd_ite(O2,A_trun_intermed,W,"mpo_A");
            AL=left_right_normalize(A_trun);
        elseif method=="OO"
            A_trun=itebd_ite(OO,AL,W,"mpo_A");#this "mpo_A" option is slightly faster when bond dimension is large
        end
        AL=left_right_normalize(A_trun);

        # vv,ee=right_eigenvector(AL,[],"A");
        # display(ee)
        # u,s,v=tsvd(permute(vv,(1,),(2,)));
        # display(sort(diag(convert(Array,s)),rev=true))
        
        #dominant eigenvalue of transfer matrix
        E=left_eigenvalue(impo_imps(O2,impo_imps(O1,AL)),AL,1)[1]
        ov=abs(left_eigenvalue(AL,AL_old,1)[1]);
        println("E="*string(E)*", "*"ov="*string(ov))
        if abs(ov-1)<1e-8
            break
        end
    
    end
    return AL,A_init
end

ITEBD_boundary_groundstate (generic function with 1 method)

In [847]:
mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());



In [848]:
#Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");


In [849]:

Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

  1.058440 seconds (4.65 M allocations: 312.983 MiB, 76.64% compilation time)
E=0.12452750731290996 - 0.00025456319378992787im, ov=0.8402070852472372
  0.465196 seconds (2.33 M allocations: 219.218 MiB, 27.14% gc time, 2.86% compilation time)
E=0.1253186989361321 + 0.00012062814930079505im, ov=0.9920601064594674
  0.336617 seconds (2.03 M allocations: 185.221 MiB)
E=0.12507376767609016 + 6.965616053598855e-5im, ov=0.9984158621223925
  0.327286 seconds (2.03 M allocations: 185.221 MiB)
E=0.1248485227564789 - 4.401287080981961e-5im, ov=0.9996288850797329
  0.441184 seconds (2.03 M allocations: 185.222 MiB, 30.10% gc time)
E=0.12471522229511219 - 0.0001441015024870821im, ov=0.9998882137719923
  0.328674 seconds (2.03 M allocations: 185.221 MiB)
E=0.12464186743605053 - 0.00022009778921314865im, ov=0.9999550533958322
  0.335214 seconds (2.03 M allocations: 185.221 MiB)
E=0.1246014224057774 - 0.00027548263618955987im, ov=0.9999773010179569
  0.329776 seconds (2.03 M allocations: 185.221 MiB)

In [850]:
#save initial CTM to compare with other codes
matwrite("itebd_matfile.mat", Dict(
    "O1" => convert(Array,O1),
    "O2" => convert(Array,O2),
    "A_init" => convert(Array,A_init),
    "Ag" => convert(Array,Ag)
); compress = false)



In [851]:
#save data
O1_dict=convert(Dict,O1);
O2_dict=convert(Dict,O2);
Ag_dict=convert(Dict,Ag);
save("itebd_data.jld", "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);




In [852]:
#load data
Ag_dict=load("itebd_data.jld")["Ag_dict"];
O1_dict=load("itebd_data.jld")["O1_dict"];
O2_dict=load("itebd_data.jld")["O2_dict"];
Ag=convert(TensorMap,Ag_dict);
O1=convert(TensorMap,O1_dict);
O2=convert(TensorMap,O2_dict);


In [853]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
display(space_AOA)
display(dim(space_AOA))
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));
display(space_AA)
display(dim(space_AA))

Rep[SU₂](0=>69, 1/2=>116, 1=>130, 3/2=>114, 2=>81, 5/2=>46, 3=>20, 7/2=>6, 4=>1)

2025

Rep[SU₂](0=>2, 1/2=>2, 1=>2, 3/2=>2, 2=>1)

25

In [854]:
# W=20;chi=20;
# (Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1)' ⊗ Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()
# Rep[SU₂](0=>4999, 1/2=>8314, 1=>9146, 3/2=>7850, 2=>5503, 5/2=>3190, 3=>1521, 7/2=>584, 4=>173, 9/2=>36, 5=>4)

In [855]:
for s in sectors(space_AOA)
  display(s)
  @show s, dim(space_AOA, s), dim(s)
end


Irrep[SU₂](0)

Irrep[SU₂](1/2)

Irrep[SU₂](1)

Irrep[SU₂](3/2)

Irrep[SU₂](2)

Irrep[SU₂](5/2)

Irrep[SU₂](3)

Irrep[SU₂](7/2)

Irrep[SU₂](4)

(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](0), 69, 1)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1/2), 116, 2)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1), 130, 3)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](3/2), 114, 4)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](2), 81, 5)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](5/2), 46, 6)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](3), 20, 7)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](7/2), 6, 8)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](4), 1, 9)


In [856]:
AOA_sec=collect(sectors(space_AOA))
AOA_sec[1]
AA_sec=collect(sectors(space_AA))
AA_sec[1]


Irrep[SU₂](0)

In [857]:
@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))


(Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [858]:
Dtrun=W*W*chi*chi;


In [976]:

euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
display(size(euL_set))
eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
display(size(eul_set))
S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
display(size(S_set))
s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
display(size(s_set))



(9,)

(5,)

(9,)

(5,)

FLR_eig: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
spin 2.5
spin 3.0
spin 3.5
spin 4.0
GLR_eig: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
FLR_svd: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
spin 2.5
spin 3.0
spin 3.5
spin 4.0
GLR_svd: 
spin 0.0
spin 0.5
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
spin 1.0
spin 1.5
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
spin 2.0


┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142


In [977]:
euL_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euL_set,evL_set,evR_set,SPIN_eig_set);
display(euL_set)
eul_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eul_set,evl_set,evr_set,spin_eig_set);
display(eul_set)
S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
display(S_set)
s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
display(s_set)

9-element Vector{Any}:
 ComplexF64[0.12453453093210645 + 0.00043135209344757897im, 0.0607290045534453 + 0.009167652753992535im, 0.029936237253014475 - 0.04957458257064171im, 0.023839949024690135 + 0.05125342066286286im, 0.047227810355477046 - 0.015052768957672132im, 0.04176702448503631 - 0.00018524744972581763im, 0.032069673925036825 + 0.012918478919439943im, 0.01056914980743253 - 0.02960893679875483im, 0.008194974344361153 + 0.030290475545543635im, 0.023354287665732398 + 0.013946874615808316im  …  0.00045423451683869893 + 0.000348149496944013im, 0.0004564653817193376 + 0.00016159316433823022im, 7.30079137320575e-5 + 0.00038571452680818437im, 0.0002943395998425062 - 3.513381482258472e-5im, -6.057766932359053e-5 - 0.00027581421568636654im, 0.00018872102785393824 + 0.00011677645031966862im, 0.0001783416044156821 + 9.477872832615533e-6im, 0.0001403622644372463 - 2.8459842335664225e-5im, 7.217244335720739e-5 - 9.351153795140126e-5im, 0.00010192624170377611 + 3.140044940150951e-5im]
 Comple

5-element Vector{Any}:
 ComplexF64[1.0000000000000007 - 2.1403838110157902e-17im, 0.5629215409166104 + 2.0164190974292193e-17im]
 ComplexF64[0.7222928584648814 + 0.30540704243075395im, 0.7222928584648813 - 0.30540704243075395im]
 ComplexF64[0.962803556587515 + 2.140325832860207e-18im, 0.6001176872621292 - 1.8297953547497148e-17im]
 ComplexF64[0.7222930341635381 + 0.33997749936971966im, 0.7222930341635381 - 0.33997749936971955im]
 ComplexF64[0.7254514848509048 + 0.0im]

9-element Vector{Any}:
 [0.14804322393725913, 0.111384585944897, 0.07492637769399231, 0.0685382484266087, 0.05627422581135422, 0.050597750112804124, 0.04804196723248351, 0.039784170929482957, 0.03776354006858249, 0.03466068605492531  …  0.0005043724277789224, 0.00038897388076155796, 0.000349695931246971, 0.00022894368507341925, 0.0002036384913241591, 0.00017537927598082497, 0.00014724255369281416, 0.00010413959074614985, 7.12992767180005e-5, 3.741240426267469e-5]
 [0.13348551816448848, 0.12769007117231507, 0.07260765449676877, 0.07154338955117154, 0.0688890311186097, 0.06249357394815057, 0.05795208124349827, 0.051531405712226294, 0.049114038868487905, 0.04863848905940167  …  0.00020837857149873254, 0.00020037995805126315, 0.0001787920585007609, 0.00016553703444571955, 0.00014532368222637535, 0.00013140983396933218, 0.00011314977476255736, 9.936151349844209e-5, 6.920275229584283e-5, 5.233414931271966e-5]
 [0.1462096372883959, 0.11387423724232656, 0.07470428771976047, 0.0690812174403523,

5-element Vector{Any}:
 [1.02913484292595, 0.5469852126628612]
 [0.7842068830453269, 0.784206882840194]
 [0.9853415326311902, 0.5863910375564355]
 [0.79830566062677, 0.7983056603594493]
 [0.7254514848509055]

In [978]:
display(space(evL_set[1][1]))
display(space(evR_set[1][1]))
display(space(evl_set[1][1]))
display(space(evr_set[1][1]))
display(space(U_set[1][1]))
display(space(Vh_set[1][1]))
display(space(u_set[1][1]))
display(space(vh_set[1][1]))



Rep[SU₂](0=>1) ← (Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)')

(Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)') ← Rep[SU₂](0=>1)

Rep[SU₂](0=>1) ← (Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)')

(Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)') ← Rep[SU₂](0=>1)

(Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)') ← Rep[SU₂](0=>1)

Rep[SU₂](0=>1) ← (Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)')

(Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)') ← Rep[SU₂](0=>1)

Rep[SU₂](0=>1) ← (Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)')

In [979]:
#check FLR_svd
@tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
M=permute(M,(1,2,3,),(4,5,6,));
M_temp=M*0;
for cc=1:length(S_set)
    S=S_set[cc];
    U=U_set[cc];
    Vh=Vh_set[cc];
    spin=SPIN_svd_set[cc]
    for cs=1:length(S)
        M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))


4.873425883226961e-15

In [980]:

#check GLR_svd
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
M_temp=M*0;
for cc=1:length(s_set)
    s=s_set[cc];
    u=u_set[cc];
    vh=vh_set[cc];
    spin=spin_svd_set[cc]
    for cs=1:length(s)
        M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

3.127809238731094e-7

In [981]:
uM,sM,vM=tsvd(M,trunc=truncdim(11));
display(sM)
display(space(uM))
M_reduced=uM*sM*vM-3*u_set[3][1]*s_set[3][1]*vh_set[3][1];

uM,sM,vM=tsvd(M_reduced,trunc=truncdim(8));
display(sM)
display(space(uM))



# #display(sort(diag(convert(Array,sM)),rev=true))
# display(abs(dot(uM,u_set[3][1])/sqrt(3)))
# display(norm(u_set[3][1]))
# display(norm(uM))

# display(abs(dot(vM,vh_set[3][1])/sqrt(3)))
# display(norm(vh_set[3][1]))
# display(norm(vM))

MM=u_set[2][1]*s_set[2][1]*vh_set[2][1]+u_set[2][2]*s_set[2][2]*vh_set[2][2];
uM,sM,vM=tsvd(MM,trunc=truncdim(11));
display(sM)
display(space(uM))

display(abs(dot(u_set[4][1],u_set[4][2])))
display(abs(dot(vh_set[4][1],vh_set[4][2])))

display(norm(uM*sM*vM-4*u_set[4][1]*s_set[4][1]*vh_set[4][1]-4*u_set[4][2]*s_set[4][2]*vh_set[4][2])/norm(uM*sM*vM))


TensorMap(ProductSpace(Rep[SU₂](1=>1, 2=>1)) ← ProductSpace(Rep[SU₂](1=>1, 2=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 0.9853415326311905
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 0.725451484850905


(Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)') ← Rep[SU₂](1=>1, 2=>1)

TensorMap(ProductSpace(Rep[SU₂](1=>1, 2=>1)) ← ProductSpace(Rep[SU₂](1=>1, 2=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 1.6454704027467271e-15
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 0.725451484850905


(Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)') ← Rep[SU₂](1=>1, 2=>1)

TensorMap(ProductSpace(Rep[SU₂](1/2=>2, 2=>1)) ← ProductSpace(Rep[SU₂](1/2=>2, 2=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2,), 1/2, (false,), ()) ← FusionTree{Irrep[SU₂]}((1/2,), 1/2, (false,), ()):
 0.3921036451975433  0.0
 0.0                 0.39210323774521727
* Data for fusiontree FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()) ← FusionTree{Irrep[SU₂]}((2,), 2, (false,), ()):
 0.0


(Rep[SU₂](1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1, 1=>1)') ← Rep[SU₂](1/2=>2, 2=>1)

3.023361614861943e-7

3.0233616127172107e-7

3.0479895058383604

In [982]:
uM,sM,vM=tsvd(M);
display(space(sM))
display(sort(diag(convert(Array,sM)),rev=true))

Rep[SU₂](0=>2, 1/2=>2, 1=>2, 3/2=>2, 2=>1) ← Rep[SU₂](0=>2, 1/2=>2, 1=>2, 3/2=>2, 2=>1)

25-element Vector{Float64}:
 1.02913484292595
 0.9853415326311905
 0.9853415326311905
 0.9853415326311905
 0.798305660307415
 0.798305660307415
 0.798305660307415
 0.798305660307415
 0.7983056603074141
 0.7983056603074141
 ⋮
 0.725451484850905
 0.725451484850905
 0.725451484850905
 0.725451484850905
 0.725451484850905
 0.5863910375564356
 0.5863910375564356
 0.5863910375564356
 0.5469852126628613

In [983]:

function truncate_sectors(n,eu_set,ev_set,ev_set2,spin_set)
    eu_full=eu_set[1]
    dim_full=2*spin_set[1].+1
    for cc=2:length(eu_set)
        eu_full=vcat(eu_full, eu_set[cc])
        dim_full=vcat(dim_full, 2*spin_set[cc].+1)
    end
    dim_full=Int.(round.(dim_full))

    order=sortperm(abs.(eu_full),rev=true);
    eu_full_sorted=eu_full[order];
    dim_full_sorted=dim_full[order];

    total_size=dim_full_sorted[1];
    old_value=eu_full_sorted[1];
    for cc=2:length(eu_full_sorted)
        total_size=total_size+dim_full_sorted[cc];
        new_value=eu_full_sorted[cc];
        if total_size>n
            break;
        end
        old_value=new_value;
    end

    for cc=1:length(eu_set)
        spins=spin_set[cc];
        eu=eu_set[cc];
        ev=ev_set[cc];
        inds=findall(x->abs(x)>=abs(old_value), abs.(eu));
        eu=eu[inds];
        ev=ev[inds];
        spins=spins[inds];
        eu_set[cc]=eu;
        ev_set[cc]=ev;
        spin_set[cc]=spins;
        if ~(ev_set2==[])
            ev2=ev_set2[cc];
            ev2=ev2[inds];
            ev_set2[cc]=ev2;
        end
    end
return eu_set,ev_set,ev_set2,spin_set

end


truncate_sectors (generic function with 2 methods)

In [984]:
function FLR_eig(A,O,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    euL_set=Vector{Any}(undef, length(sector_set));
    evL_set=Vector{Any}(undef, length(sector_set));
    euR_set=Vector{Any}(undef, length(sector_set));
    evR_set=Vector{Any}(undef, length(sector_set));
    println("FLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))

        FLR_eig_L(x)=HV_FL(x,A,O);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-16));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(euL)==sort(abs.(euL), rev=true)
        euL=euL[1:minimum([length(euL),n_eff,dim(full_space,sec)])];
        evL=evL[1:minimum([length(evL),n_eff,dim(full_space,sec)])];
        euL_set[cc]=euL;
        evLL=evL;
        evL=Vector{Any}(undef, length(evLL));
        for cs=1:length(evL)
            evL[cs]=permute(evLL[cs],(1,),(2,3,4,))
        end
        evL_set[cc]=evL;
        SPIN_set[cc]=ones(length(euL))*(dim(sec)-1)/2;
        
        FLR_eig_R(x)=HV_FR(x,A,O);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-16));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(euR)==sort(abs.(euR), rev=true)
        euR=euR[1:minimum([length(euR),n_eff,dim(full_space,sec)])];
        evR=evR[1:minimum([length(evR),n_eff,dim(full_space,sec)])];
        euR_set[cc]=euR;
        evRR=evR;
        evR=Vector{Any}(undef, length(evRR));
        for cs=1:length(evR)
            evR[cs]=permute(evRR[cs],(1,2,3,),(4,))
        end
        evR_set[cc]=evR;

        @assert (norm(abs.(euL)-abs.(euR))/norm(euL))<1e-8
    end
    return euL_set,evL_set,euR_set,evR_set,SPIN_set     
end
        
function HV_FL(vl,A,mpo)
    @tensor vl[:]:=vl[-1,1,3,5]*A'[1,-2,2]*mpo[3,4,-3,2]*A[5,-4,4];
    return vl
end

function HV_FR(vr,A,mpo)
    @tensor vr[:]:=A'[-1,1,2]*mpo[-2,4,3,2]*A[-3,5,4]*vr[1,3,5,-4];
    return vr
end
 
        


HV_FR (generic function with 1 method)

In [986]:
function GLR_eig(A,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    eul_set=Vector{Any}(undef, length(sector_set));
    evl_set=Vector{Any}(undef, length(sector_set));
    eur_set=Vector{Any}(undef, length(sector_set));
    evr_set=Vector{Any}(undef, length(sector_set));
    println("GLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))
        
        GLR_eig_L(x)=HV_L_tensor(x,A,[]);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-16));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(eul)==sort(abs.(eul), rev=true)
        
        eul=eul[1:minimum([length(eul),n_eff,dim(full_space,sec)])];
        evl=evl[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eul_set[cc]=eul;
        evll=evl;
        evl=Vector{Any}(undef, length(evll));
        for cs=1:length(evl)
            evl[cs]=permute(evll[cs],(1,),(2,3,))
        end
        evl_set[cc]=evl;
        spin_set[cc]=ones(length(eul))*(dim(sec)-1)/2;

        GLR_eig_R(x)=HV_R_tensor(x,A,[]);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-16));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(eur)==sort(abs.(eur), rev=true)
        eur=eur[1:minimum([length(eur),n_eff,dim(full_space,sec)])];
        evr=evr[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eur_set[cc]=eur;
        evrr=evr;
        evr=Vector{Any}(undef, length(evrr));
        for cs=1:length(evr)
            evr[cs]=permute(evrr[cs],(1,2,),(3,))
        end
        evr_set[cc]=evr;

        @assert (norm(abs.(eul)-abs.(eur))/norm(eul))<1e-8

    end
    return eul_set,evl_set,eur_set,evr_set,spin_set   
end
    
    
    

GLR_eig (generic function with 1 method)

In [987]:
function FLR_svd(A,O,pow,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    S_set=Vector{Any}(undef, length(sector_set));
    U_set=Vector{Any}(undef, length(sector_set));
    Vh_set=Vector{Any}(undef, length(sector_set));
    println("FLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))

        FLR_svd_R(x)=HV_FR_pow(x,A,O,pow);
        FLR_svd_R_conj(x)=HV_FR_conj_pow(x,A,O,pow);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of singular values obtained are not enough, use smaller tol")
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-16);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(S)==sort(abs.(S), rev=true)
        S=S[1:minimum([length(S),n_eff,dim(full_space,sec)])];
        U=U[1:minimum([length(U),n_eff,dim(full_space,sec)])];
        V=V[1:minimum([length(V),n_eff,dim(full_space,sec)])];

        UU=U;
        U=Vector{Any}(undef, length(UU));
        for cs=1:length(U)
            U[cs]=permute(UU[cs],(1,2,3,),(4,))
        end

        Vh=Vector{Any}(undef, length(V));
        for cs=1:length(Vh)
            vvr=V[cs];
            Vh[cs]=permute(vvr',(4,),(1,2,3,));
        end
        S_set[cc]=S;
        U_set[cc]=U;
        Vh_set[cc]=Vh;
        SPIN_set[cc]=ones(length(S))*(dim(sec)-1)/2;
        
    end
    return S_set,U_set,Vh_set,SPIN_set     
end
            

function HV_FR_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR(vr,A,mpo);
    end
    return vr
end
function HV_FR_conj(vr,A,mpo)
    @tensor vr[:]:=A[1,-1,2]*mpo'[3,4,-2,2]*A'[5,-3,4]*vr[1,3,5,-4];
    return vr
end
function HV_FR_conj_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR_conj(vr,A,mpo);
    end
    return vr
end
    

HV_FR_conj_pow (generic function with 1 method)

In [988]:
function GLR_svd(A,pow,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    s_set=Vector{Any}(undef, length(sector_set));
    u_set=Vector{Any}(undef, length(sector_set));
    vh_set=Vector{Any}(undef, length(sector_set));
    A_noise=TensorMap(randn, codomain(A),domain(A));
    A_noise=A_noise/norm(A_noise)*norm(A)*1e-9;
    println("GLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))


        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise*0);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise*0);
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])    
        catch e
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise);
            println("Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol")
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*4, tol=1e-14);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(s)==sort(abs.(s), rev=true)
    
        s=s[1:minimum([length(s),n_eff,dim(full_space,sec)])];
        u=u[1:minimum([length(u),n_eff,dim(full_space,sec)])];
        v=v[1:minimum([length(v),n_eff,dim(full_space,sec)])];

        uu=u;
        u=Vector{Any}(undef, length(u));
        for cs=1:length(u)
            u[cs]=permute(uu[cs],(1,2,),(3,))
        end

        vh=Vector{Any}(undef, length(v));
        for cs=1:length(vh)
            vvr=v[cs];
            vh[cs]=permute(vvr',(3,),(1,2,));
        end
        s_set[cc]=s;
        u_set[cc]=u;
        vh_set[cc]=vh;
        spin_set[cc]=ones(length(s))*(dim(sec)-1)/2;
        
    end
    return s_set,u_set,vh_set,spin_set     
end

function HV_R_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
    end
    return vr
end

function HV_R_conj_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_[1,-1,2]*A'[3,-2,2]*vr[1,3,-3];
    end
    return vr
end



HV_R_conj_pow (generic function with 2 methods)

In [989]:

function TransfOp_decom(A,O,pow,Dtrun_init,Dtrun_max,trun_tol,method)
    Dstep=50;
    Dtrun=Dtrun_init;
    if method=="eigenvalue_FLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            evL,lambdaL=F_L(A,O,Dtrun);
            evR,lambdaR=F_R(A,O,Dtrun);
    
            eu=abs(lambdaL);
            eu=eu/max(eu); 
            eu=eu.^pow;
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal eigenvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)*(size(O,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        evL=evL(:,order);
        lambdaL=lambdaL(order);
        evR=evR(:,order);
        lambdaR=lambdaR(order);
        
        posit=find(eu_sorted>trun_tol);
        evL=evL(:,posit);
        lambdaL=lambdaL(posit);
        evR=evR(:,posit);
        lambdaR=lambdaR(posit);
        
        return evL,lambdaL,evR,lambdaR

    elseif method=="eigenvalue_GLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            evl,lambdal=G_L(A,A,Dtrun);
            evr,lambdar=G_R(A,A,Dtrun);
    
            eu=abs(lambdal);
            eu=eu/max(eu); 
            eu=eu.^pow;
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal eigenvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>size(A,1)^2
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        evl=evl(:,order);
        lambdal=lambdal(order);
        evr=evr(:,order);
        lambdar=lambdar(order);
        
        posit=find(eu_sorted>trun_tol);
        evl=evl(:,posit);
        lambdal=lambdal(posit);
        evr=evr(:,posit);
        lambdar=lambdar(posit);
        
        return evl,lambdal,evr,lambdar
    elseif method=="svd_GLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            [u,s,v]=GLR_svd(A,pow,Dtrun);
            vt=v';
            eu=abs(diag(s));
            eu=eu/max(eu); 
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal singularvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        u=u(:,order);
        s=s(order,order);
        vt=vt(order,:);
    
        posit=find(eu_sorted>trun_tol);
        u=u(:,posit);
        s=s(posit,posit);
        vt=vt(posit,:);
        
        s=diag(s);
        return u,s,vt
    elseif method=="svd_FLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            [U,S,V]=FLR_svd(A,O,pow,Dtrun);
            Vt=V';   
            eu=abs(diag(S));
            eu=eu/max(eu); 
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal singularvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)*(size(O,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        U=U(:,order);
        S=S(order,order);
        Vt=Vt(order,:);
        
        posit=find(eu_sorted>trun_tol);
        U=U(:,posit);
        S=S(posit,posit);
        Vt=Vt(posit,:);
        
        
        S=diag(S);
        
        return U,S,Vt
    end
end
    

    
    


ErrorException: syntax: character literal contains multiple characters

In [990]:

    function GLR_svd_test(A,pow,n,full_space,sector_set)
        spin_set=Vector{Any}(undef, length(sector_set));
        s_set=Vector{Any}(undef, length(sector_set));
        u_set=Vector{Any}(undef, length(sector_set));
        vh_set=Vector{Any}(undef, length(sector_set));
        A_noise=TensorMap(randn, codomain(A),domain(A));
        A_noise=A_noise/norm(A_noise)*norm(A)*1e-9;
        println("GLR_svd: ")
        for cc=1:length(sector_set)
            sec=sector_set[cc];
            spin=(dim(sec)-1)/2;
            n_eff=Int(round(n/dim(sec)))+1
            println("spin "*string(spin))
    

            vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
            try
                GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise*0);
                GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise*0);
                s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2);
                @assert info.converged >= minimum([n_eff,dim(full_space,sec)])    
            catch e
                GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise);
                GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise);
                println("Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol")
                s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*4, tol=1e-14);
                @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
            end
            @assert abs.(s)==sort(abs.(s), rev=true)
        
            s=s[1:minimum([length(s),n_eff,dim(full_space,sec)])];
            u=u[1:minimum([length(u),n_eff,dim(full_space,sec)])];
            v=v[1:minimum([length(v),n_eff,dim(full_space,sec)])];
    
            uu=u;
            u=Vector{Any}(undef, length(u));
            for cs=1:length(u)
                u[cs]=permute(uu[cs],(1,2,),(3,))
            end
    
            vh=Vector{Any}(undef, length(v));
            for cs=1:length(vh)
                vvr=v[cs];
                vh[cs]=permute(vvr',(3,),(1,2,));
            end
            s_set[cc]=s;
            u_set[cc]=u;
            vh_set[cc]=vh;
            spin_set[cc]=ones(length(s))*(dim(sec)-1)/2;
            
        end
        return s_set,u_set,vh_set,spin_set     
    end
    
    function HV_R_pow(vr,A,pow,A_noise)
        A_=A+A_noise;
        for cc=1:pow
            @tensor vr[:]:=A_'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
        end
        return vr
    end
    
    function HV_R_conj_pow(vr,A,pow,A_noise)
        A_=A+A_noise;
        for cc=1:pow
            @tensor vr[:]:=A_[1,-1,2]*A'[3,-2,2]*vr[1,3,-3];
        end
        return vr
    end
    
    

HV_R_conj_pow (generic function with 2 methods)

In [992]:
# A_dense=convert(Array,A);
# @tensor AA[:]:=conj(A_dense)[-1,-3,1]*A_dense[-2,-4,1];
# AA=reshape(AA,size(AA,1)*size(AA,2),size(AA,3)*size(AA,4));
# eigvals(AA)


In [994]:

s_set,u_set,vh_set,spin_svd_set=GLR_svd_test(Ag,pow,Dtrun,space_AA,AA_sec);
display(s_set)


5-element Vector{Any}:
 [1.02913484292595, 0.5469852126628613]
 [0.7842068829258476, 0.784206882034862]
 [0.9853415326311906, 0.5863910375564354]
 [0.798305660510277, 0.7983056595789064]
 [0.7254514848509055]

GLR_svd: 
spin 0.0
spin 0.5
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
spin 1.0
spin 1.5
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
spin 2.0


┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142


In [995]:
#check GLR_svd
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
M_temp=M*0;
for cc=1:length(s_set)
    s=s_set[cc];
    u=u_set[cc];
    vh=vh_set[cc];
    spin=spin_svd_set[cc]
    for cs=1:length(s)
        M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

3.07399168016545e-7